In [12]:
import numpy as np 
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.datasets import fetch_mldata
import random 
import math 
import pandas as pd
import xlrd
import matplotlib.pyplot as plt

In [7]:
iris = load_iris()
target = np.zeros((iris['target'].shape))
for i in range (len(iris['target'])):
    if iris['target'][i] == 0:
        target[i] = 1
    else :
        target[i] = 0 
features = iris['data'][:,[0,3]]

In [8]:
def sigmoid_func(x,theta):
    num = 1
    mult = x.dot(theta)
    den = 1 + math.exp(-mult)
    return num/den

def log_reg(X,Y,iterations = 10, alpha = 0.1):
    r,c = X.shape
    x = np.c_[np.ones(r),X]
    r,c = x.shape
    theta = np.zeros((c,1))
    for i in range (iterations):
        for k in range(c):
            derivative = 0 
            for j in range(0,r):
                derivative += (sigmoid_func(x[j,:],theta) - Y[j])*x[j,k]
            theta[k,0] = theta[k,0] - alpha*derivative
    return theta
        

In [9]:
r,c = features.shape
x = np.c_[np.ones(r),features]
theta = log_reg(features,target)
predictions = np.zeros((target.shape))
for i in range (target.shape[0]):
    pred = sigmoid_func(x[i,:],theta)
    if pred>=0.5:
        predictions[i] = 1
    else :
        predictions[i] = 0
predictions

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
train_X,train_Y = shuffle(features,target,random_state = 42)
clf = LogisticRegression()
clf.fit(train_X,train_Y)
clf.predict(features)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

# NEURAL NETWORKS

In [625]:
class NeuralNetwork():
    def __init__(self,x,y,hiddenlayers,activations,epochs,alpha):
        self.input = x
        self.y = y
        self.activations = activations
        self.epochs = epochs
        self.alpha = alpha
        self.weights= {} 
        self.weights['1'] = np.random.rand(self.input.shape[1],hiddenlayers[0])
        self.z_layers ={}
        self.der_lin = None
        self.der_relu = None
        self.der_sig = None
        #self.der_act = 
        for i in range (len(hiddenlayers)-1):
            self.weights[str(i+2)] = np.random.rand(hiddenlayers[i],hiddenlayers[i+1])
        self.output = np.zeros((self.y.shape))
       
        def der_relu(x):
            row,col = np.shape(x)
            t = np.copy(x)
            for i in range(row):
                for j in range(col):
                    if t[i,j]>0:
                        t[i,j]=1
                    else:
                        t[i,j]=0
            return t
        self.der_relu = der_relu
       
        def der_sig(x):
            return (1./(1+np.exp(-x)))*(1-(1./(1+np.exp(-x))))
        self.der_sig = der_sig
       
        def der_lin(x):
            return np.ones(x.shape)
        self.der_lin = der_lin
        self.der_act = {'relu': self.der_relu, 'sigmoid' : self.der_sig, 'linear' : self.der_lin}
            
    def sigmoid(self,x):
        return 1./(1+np.exp(-x))
    
    def ReLU(self,x):
        return np.maximum(0,x)
    
    def act_layer(self,i,x):
        if self.activations[i-1]== 'relu':
            return self.ReLU(x)
        elif self.activations[i-1] == 'sigmoid':
            return self.sigmoid(x)
        else :
            return x
    
    def feedforward(self):
        self.act_layers = {}
        self.z_layers = {}
        self.act_layers['1'] = self.act_layer(1,np.matmul(self.input,self.weights['1']))
        self.z_layers['1'] = np.dot(self.input,self.weights['1'])
        for k in range (len(self.activations)-1):
            self.z_layers[str(k+2)] =  np.dot(self.act_layers[str(k+1)],self.weights[str(k+2)])
            self.act_layers[str(k+2)] = self.act_layer(k+1,self.z_layers[str(k+2)])
        self.output = self.act_layers[str(len(self.activations))]
        return self.output
    
    def backpropagation(self):
        self.dw = {}
        self.errors = {}
        self.errors[str(len(self.activations))] = (self.y - self.output)* self.der_act[self.activations[-1]](self.z_layers[str(len(self.activations))])
        for i in reversed(range(len(self.activations)-1)):
            
            self.errors[str(i+1)] = (self.errors[str(i+2)].dot((self.weights[str(i+2)]).T))*(self.der_act[self.activations[i]](self.z_layers[str(i+1)]))
        self.dw['1'] = (self.input.T).dot(self.errors['1'])
        for i in range(len(self.weights)-1):
            self.dw[str(i+2)] = (self.act_layers[str(i+1)].T).dot(self.errors[str(i+2)])
        return(self.z_layers['1'].shape)                                                          
  
    
    
    def train(self):
        for _ in range (self.epochs):
            nn.feedforward()
            nn.backpropagation()
            mse_loss = (np.sum((self.y - self.output)*(self.y-self.output)))/len(self.y)
            abs_loss = (np.sum((self.y - self.output)))/len(self.y)
            for i in range(len(self.weights)):
                self.weights[str(i+1)] = self.weights[str(i+1)] - (self.alpha)*(self.dw[str(i+1)]) 
            print('rmse loss :' ,math.sqrt(mse_loss))
            print( 'abs_loss :', abs(abs_loss))
    
    def test(self,test_X,test_Y):
        act_layers = {}
        z_layers = {}
        act_layers['1'] = self.act_layer(1,np.dot(test_X,self.weights['1']))
        z_layers['1'] = np.dot(test_X,self.weights['1'])
        for k in range (len(self.activations)-1):
            z_layers[str(k+2)] =  np.dot(act_layers[str(k+1)],self.weights[str(k+2)])
            act_layers[str(k+2)] = self.act_layer(k+2,z_layers[str(k+2)])
        output = act_layers[str(len(self.activations))]
        print(z_layers['2'].shape)
        mse_loss = (np.sum((test_Y - output)*(test_Y - output)))/len(test_Y)
        abs_loss = (np.sum((test_Y - output)))/len(test_Y)
        print('rmse loss :' ,math.sqrt(mse_loss))
        print( 'abs_loss :', abs(abs_loss))
    

In [626]:
r_est_price1=pd.read_excel('Real_estate.xlsx')

In [627]:
r_est_price=r_est_price1.loc[:,['X1 transaction date', 'X2 house age',
       'X3 distance to the nearest MRT station',
       'X4 number of convenience stores', 'X5 latitude', 'X6 longitude',
       'Y house price of unit area']]
r_est_price.columns=['X1', 'X2',
       'X3',
       'X4', 'X5', 'X6',
       'Y']

In [628]:
Y = r_est_price.values[:,6].reshape(-1,1)
X = r_est_price.values[:,0:5]
from sklearn.model_selection import train_test_split
train_X,test_X,train_Y,test_Y = train_test_split(X,Y,test_size = 0.3,random_state = 42)
nn = NeuralNetwork(train_X,train_Y,[4,2,1],['sigmoid','sigmoid','linear'],100,0.1)
nn.feedforward()
nn.backpropagation()
nn.train()

rmse loss : 40.218465126028875
abs_loss : 37.76225406320998
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss : 37.94186851211073
rmse loss : 40.387157505195646
abs_loss 

In [624]:
nn.test(test_X,test_Y)

(125, 2)
rmse loss : 39.11240110246366
abs_loss : 36.91280000000001


In [3]:
from sklearn.model_selection import train_test_split
mnist = fetch_mldata('MNIST original')
X = mnist.data
Y = mnist.target
train_X,test_X,train_Y,test_Y = train_test_split(X,Y,test_size = 0.3,random_state = 42)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
class NeuralNetwork_clf():
    def __init__(self,x,y,hiddenlayers,activations,epochs,alpha):
        self.input = x
        self.y = y
        self.activations = activations
        self.epochs = epochs
        self.alpha = alpha
        self.weights= {} 
        self.weights['1'] = np.random.rand(self.input.shape[1],hiddenlayers[0])
        self.z_layers ={}
        self.der_lin = None
        self.der_relu = None
        self.der_sig = None
        #self.der_act = 
        for i in range (len(hiddenlayers)-1):
            self.weights[str(i+2)] = np.random.rand(hiddenlayers[i],hiddenlayers[i+1])
        self.output = np.zeros((self.y.shape))
       
        def der_relu(x):
            row,col = np.shape(x)
            t = np.copy(x)
            for i in range(row):
                for j in range(col):
                    if t[i,j]>0:
                        t[i,j]=1
                    else:
                        t[i,j]=0
            return t
        self.der_relu = der_relu
       
        def der_sig(x):
            return (1./(1+np.exp(-x)))*(1-(1./(1+np.exp(-x))))
        self.der_sig = der_sig
       
        def der_lin(x):
            return np.ones(x.shape)
        self.der_lin = der_lin
        self.der_act = {'relu': self.der_relu, 'sigmoid' : self.der_sig, 'linear' : self.der_lin}
            
    def sigmoid(self,x):
        return 1./(1+np.exp(-x))
    
    def ReLU(self,x):
        return np.maximum(0,x)
    
    def act_layer(self,i,x):
        if self.activations[i-1]== 'relu':
            return self.ReLU(x)
        elif self.activations[i-1] == 'sigmoid':
            return self.sigmoid(x)
        else :
            return x
    
    def feedforward(self):
        self.act_layers = {}
        self.z_layers = {}
        self.act_layers['1'] = self.act_layer(1,np.dot(self.input,self.weights['1']))
        self.z_layers['1'] = np.dot(self.input,self.weights['1'])
        for k in range (len(self.activations)-1):
            self.z_layers[str(k+2)] =  np.dot(self.act_layers[str(k+1)],self.weights[str(k+2)])
            self.act_layers[str(k+2)] = self.act_layer(k+1,self.z_layers[str(k+2)])
        self.output = self.act_layers[str(len(self.activations))]
        return self.output
    
    def backpropagation(self):
        self.dw = {}
        self.errors = {}
        self.errors[str(len(self.activations))] = (-(self.y/self.output)+(1-self.y)/(1-self.output))* self.der_act[self.activations[-1]](self.z_layers[str(len(self.activations))])
        for i in reversed(range(len(self.activations)-1)):
            
            self.errors[str(i+1)] = (self.errors[str(i+2)].dot((self.weights[str(i+2)]).T))*(self.der_act[self.activations[i]](self.z_layers[str(i+1)]))
        self.dw['1'] = (self.input.T).dot(self.errors['1'])
        for i in range(len(self.weights)-1):
            self.dw[str(i+2)] = (self.act_layers[str(i+1)].T).dot(self.errors[str(i+2)])
        return(self.z_layers['1'].shape)                                                          
  
    
    
    def train(self):
        for _ in range (self.epochs):
            nn.feedforward()
            nn.backpropagation()
            mse_loss = (np.sum((self.y - self.output)*(self.y-self.output)))/len(self.y)
            abs_loss = (np.sum((self.y - self.output)))/len(self.y)
            for i in range(len(self.weights)):
                self.weights[str(i+1)] = self.weights[str(i+1)] - (self.alpha)*(self.dw[str(i+1)]) 
            print('rmse loss :' ,math.sqrt(mse_loss))
            print( 'abs_loss :', abs(abs_loss))
    
    def test(self,test_X,test_Y):
        act_layers = {}
        z_layers = {}
        act_layers['1'] = self.act_layer(1,np.dot(test_X,self.weights['1']))
        z_layers['1'] = np.dot(test_X,self.weights['1'])
        for k in range (len(self.activations)-1):
            z_layers[str(k+2)] =  np.dot(act_layers[str(k+1)],self.weights[str(k+2)])
            act_layers[str(k+2)] = self.act_layer(k+1,self.z_layers[str(k+2)])
        output = self.act_layers[str(len(self.activations))]
        mse_loss = (np.sum((test_Y - output)*(test_Y - output)))/len(test_Y)
        abs_loss = (np.sum((test_Y - output)))/len(test_Y)
        print('rmse loss :' ,math.sqrt(mse_loss))
        print( 'abs_loss :', abs(abs_loss))

In [5]:
nn = NeuralNetwork_clf(train_X,train_Y,[4,2,1],['sigmoid','sigmoid','linear'],100,0.1)
nn.feedforward()
nn.backpropagation()
#nn.train()
#nn.test(test_X,test_Y)

MemoryError: 

reference:https://medium.com/@a.mirzaei69/implement-a-neural-network-from-scratch-with-python-numpy-backpropagation-e82b70caa9bb 